# Libraries

In [1]:
import numpy as np
from docx import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


c:\Users\Juanes\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm-4.66.2-py3.11.egg\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
def top_similarities(embeddings, query_vector):
    """
    Calculate the top similar vectors using cosine distance between a list of vectors and a query vector.

    Parameters:
    - embeddings (numpy.ndarray): Matrix containing the list of vectors.
    - query_vector (numpy.ndarray): Vector to compare against the list of vectors.

    Returns:
    - Tuple[int, int]: Indices of the top two vectors with the highest cosine similarity with the query vector.
    """
    # Convert the input vector and the list of vectors to NumPy matrices for compatibility with cosine_similarity
    vectors_matrix = np.vstack([embeddings, query_vector])

    # Calculate similarity using cosine distance
    similarities = cosine_similarity(vectors_matrix)

    # Get similarities with the last vector (the input vector)
    similarities_with_input_vector = similarities[-1, :-1]

    # Get the top two indices with highest similarities
    top_indices = np.argsort(similarities_with_input_vector)[-10:]

    # Return the top two indices
    return list(top_indices)[::-1]



In [3]:

def read_word_file(file_path):
    """
    Read the content of a Word file and return it as a string.
    """
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def split_by_periods(text):
    """
    Split a text into a list at each period.
    """
    return text.split(".")





In [4]:
def process_sentences(sentences):
    """
Processes a list of sentences by adjusting their lengths to improve the coherence of the statements.

Parameters:

sentences (list): List of strings representing sentences.
Returns:

list: List of processed sentences, where long sentences have been shortened, and short ones concatenated.
"""
    # Get the length of each sentence
    sentence_lengths = [len(each) for each in sentences]
    # Determine longest outlier
    long = np.mean(sentence_lengths) + np.std(sentence_lengths)

    # Determine shortest outlier
    short = np.mean(sentence_lengths) - np.std(sentence_lengths)

    # Shorten long sentences and concatenate short ones
    processed_sentences = []
    for each in sentences:
        if len(each) > long:
            # Let's replace all the commas with dots
            comma_splitted = each.replace(',', '.')
            # Split the long sentence by dots and extend the result to the processed sentences list
            processed_sentences.extend(comma_splitted.split('. '))
        elif len(each) < short:
            # Concatenate short sentences
            processed_sentences[-1] += f' {each}' if processed_sentences else f'{each}'
        else:
            processed_sentences.append(each)

    return processed_sentences


# File Loading and Model Application

In [5]:
# Path to the Word file
word_file_path = "texto_embeddings.docx"

# Read the content of the Word file
file_text = read_word_file(word_file_path)

# Split the text by periods
sentences = split_by_periods(file_text)

In [6]:
# Process the sentences using the process_sentences function
new_sentences = process_sentences(sentences)

# Encode the processed sentences using the model
embeddings2 = model.encode(new_sentences)

# Print the lengths of the original and processed sentences
print("Original sentences:", len(sentences))
print("Processed sentences:", len(new_sentences))

Original sentences: 40
Processed sentences: 52


In [7]:

from http.server import HTTPServer, BaseHTTPRequestHandler
from urllib.parse import urlparse, parse_qs
import os
import io
import cgi



# HTTP Server definition Class, itis intended just to recive POSTs requests
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):

    def do_POST(self):
        print("Peticion recibida")
        content_type, pdict = cgi.parse_header(self.headers['content-type'])

        if content_type == 'multipart/form-data':
            # Parse the form data
            pdict['boundary'] = pdict['boundary'].encode('ascii')  # Ensure it's bytes
            form_data = cgi.parse_multipart(self.rfile, pdict)

            # Extract the field values (without double quotes)
            question = form_data['question'][0]



        # create a dataset to allocate the iris params
        print("Pregunta:", question)
        lista_texto = [question]
        question = model.encode(lista_texto)
        response_index = top_similarities(embeddings2, question)
        print(response_index)
        for i in response_index:
            print(new_sentences[i])
        # Construct HTML response
        html_response = f"""
        <html>
        <head><title>Respuesta a la pregunta</title></head>
        <body>
            <h2>Respuesta a la pregunta</h2>
            <p> {new_sentences[response_index[0]]}</p>
        </body>
        </html>
        """

        # HTTP request response
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.send_header("Access-Control-Allow-Origin", "*")
        self.end_headers()
        self.wfile.write(html_response.encode('utf-8'))

#Server init over the 8000 port
print("Iniciando el servidor...")
server = HTTPServer(('localhost', 8000), SimpleHTTPRequestHandler)
server.serve_forever()

Iniciando el servidor...


C:\Users\Juanes\AppData\Local\Temp\ipykernel_2712\2511772143.py:5: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


Peticion recibida
Pregunta: cuantos asistentes llegan de los continentes
[2, 10, 32, 41, 7, 8, 20, 50, 28, 26]

Cada año más de 90000 asistentes (entre expositores, inversionistas, periodistas y apasionados por la tecnología en general) llegan desde los cinco continentes para hablar de lo que será tendencia en los próximos años
generar conciencia en los turistas

Para los organizadores de este congreso

Cifras manejadas por la GSMA dan cuenta de que en el mundo unos 400 operadores, en 116 países, han lanzado servicios de eSIM, lo que se traduce en un aumento considerable y un impulso para este mercado
pues la región atraviesa por una sequía que podría agravarse debido a la demanda de agua que generarán estas decenas de miles de visitantes

Sobre esto último los organizadores del evento han emprendido una alianza con la red de hoteles de Barcelona para
y el evento reunirá a industrias
mencionan los organizadores del MWC

Los asistentes no solo podrán apreciar un ejemplar de este vehícul

127.0.0.1 - - [29/Feb/2024 21:12:53] "POST /predict HTTP/1.1" 200 -


Peticion recibida
Pregunta: cuanto dinero deja a barcelona
[3, 8, 25, 4, 10, 11, 26, 36, 42, 20]

Un evento que en 2023 derramó en la economía de Barcelona y Cataluña más de 461 millones de euros (cerca de 2 billones de pesos colombianos) y creó unos 9100 puestos de trabajo a tiempo parcial

Sobre esto último los organizadores del evento han emprendido una alianza con la red de hoteles de Barcelona para
según lo explicado por la compañía
los cuales se suman a los 150000 que se estima se han generado desde 2006
generar conciencia en los turistas
así como emplear mecanismos de ahorro y reutilización del líquido
está pensado para transportar a sus ocupantes (cuatro pasajeros y un piloto) de forma rápida (a más de 300 kilómetros por hora) y silenciosa (100 menos ruidoso que un helicóptero) en ciudades congestionadas
la verdad es que su penetración e impacto en la sociedad ha sido más bien escaso (especialmente por su costo y falta de desarrollos que verdaderamente sean disruptivos)
 No obs

...

Peticion recibida
Pregunta: tecnologia 5G
[13, 12, 15, 40, 37, 0, 18, 16, 31, 33]
 Casos de uso en materia de conectividad en teléfonos inteligentes, dispositivos IoT para las empresas y el hogar, así como la conducción autónoma, han sido parte los principales temas de conversación  Hay quienes ya hablan de lo que será el 6G y el 7G, así como de su impacto en la industria

En los últimos años, la conectividad 5G ha sido la protagonista en los pasillos de la Fira, que es el centro de convenciones en donde se desarrolla este congreso
 En esa línea, las conferencias del MWC estarán concentradas en seis subtemas: incluidos 5G y más allá, Conectando todo, Humanizando la IA, Fabricación DX, Game Changers y Nuestro ADN digital

Otro de los vaticinios para los próximos años es la acogida de las eSIM, una tecnología que le permite a los teléfonos tener conectividad móvil sin la necesidad de instalar un chip de telefonía
 Sin embargo, esta es una tecnología a la que todavía se le sigue apostando

127.0.0.1 - - [29/Feb/2024 21:14:29] "POST /predict HTTP/1.1" 200 -


Peticion recibida
Pregunta: cuantos operadores usan eSIM
[41, 40, 38, 42, 25, 50, 9, 34, 8, 12]

Cifras manejadas por la GSMA dan cuenta de que en el mundo unos 400 operadores, en 116 países, han lanzado servicios de eSIM, lo que se traduce en un aumento considerable y un impulso para este mercado

Otro de los vaticinios para los próximos años es la acogida de las eSIM, una tecnología que le permite a los teléfonos tener conectividad móvil sin la necesidad de instalar un chip de telefonía

“los operadores y el ecosistema móvil han estado ayudando a dar forma al panorama del entretenimiento durante años
 No obstante, estas cifras también revelan que los el grueso de los operadores aún no se subido al bus de esta transformación, por lo que la capacidad de crecimiento de esta industria sigue siendo considerable
según lo explicado por la compañía
mencionan los organizadores del MWC
mediante estrategias informativas
ofreciendo con esto grandes oportunidades en el retorno de la inversión y n

127.0.0.1 - - [29/Feb/2024 21:15:45] "POST /predict HTTP/1.1" 200 -


: 

The quality and relevance of the obtained answers largely depend on the coherence between the formulated questions and the knowledge embedded in the model. It is essential that the person generating the questions is familiar with the content of the sentences added to the model to ensure consistent and meaningful results.